In [2]:
import sys
import os

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

블로그 리뷰 url 크롤링

In [ ]:
# 여러 동에 대한 반복문
dong_list = ['연희동', '후암동', '망원동', '연남동', '성수동']
for dong in dong_list:
    # 각 동에 해당하는 카페 리스트 파일 읽기
    cafe_list_path = f'/Users/hana/MJU/3-2/빅데이터프로그래밍/BDP_Proj/cafelist/cafelist_{dong}.csv'
    cafe_list = pd.read_csv(cafe_list_path)
    cafe_list = cafe_list.drop("Unnamed: 0", axis=1)
    cafe_list = cafe_list.drop("cafe_type", axis=1)
    
    for cafe in cafe_list["name"]:
        dong_name = dong.replace('동', '')
        if dong_name + '점' in cafe:
            cafe = cafe.replace(dong_name + '점', '')
        
        cafe = dong_name + " " + cafe
        cafe_keywords = cafe.split()
    
        # 파일 경로 설정
        folder_path = f"BDP-Project/{dong}/"
        file_path = os.path.join(folder_path, f"{cafe}_블로그_리뷰.csv")
        if not os.path.exists(file_path):

            #크롬 웹브라우저 실행
            chrome_options = webdriver.ChromeOptions()
            
            chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")
            driver = webdriver.Chrome()
            driver.get(
                "https://search.naver.com"
            )
            time.sleep(1)
            
            cafe_keywords = cafe.split()
    
            #검색창에 "검색어" 검색
            element = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            element.send_keys(dong+" "+cafe)
            element.submit()
            time.sleep(1)
        
            #VIEW 클릭
            driver.find_element(By.LINK_TEXT, "VIEW").click()
            
            #옵션 클릭
            driver.find_element(By.LINK_TEXT, "옵션").click()
            
            #블로그 클릭
            driver.find_element(By.LINK_TEXT, "블로그").click()
        
            url_list = []
            title_list = []
        
            # 스크롤을 밑으로 내려주는 함수
            def scroll_down(driver):
                driver.execute_script("window.scrollTo(0, 99999999)")
                time.sleep(1)
            
                # n : 스크롤할 횟수 설정
                n = 5 #스크롤 1번당 글 30개씩 화면에 출력
                i = 0
                while i < n:
                    scroll_down(driver) #스크롤다운
                    i = i+1
            
            
            #URL_raw 크롤링
            articles = "a.title_link"
            article_raw = driver.find_elements(By.CSS_SELECTOR, articles)
            for article in article_raw:
                title = article.text
        
                if all(keyword in title for keyword in cafe_keywords):
                    url = article.get_attribute('href')
                    url_list.append(url)
                    title_list.append(title)
        
                else:
                    continue
                    
            # 10개 미만으로 게시글을 가질 경우 검색 결과 상위 10개 추출
            if (len(url_list) < 10):
                url_list = []
                title_list = []

                article_raw = article_raw[:10]
                
                #URL_raw 크롤링
                for article in article_raw:
                    url = article.get_attribute('href')
                    url_list.append(url)
                
                    title = article.text
                    title_list.append(title)

            print("")
            print(cafe)
            print('url 갯수: ',len(url_list))
            print('title 갯수: ',len(title_list))
            
            df = {}
            df = pd.DataFrame({'url':url_list, 'title':title_list})
            
            # 폴더가 존재하지 않으면 생성
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
            file_name = f"{cafe}_블로그_리뷰.csv"
            
            # 파일 전체 경로 생성
            file_path = os.path.join(folder_path, file_name)
            
            # 데이터프레임을 CSV 파일로 저장
            df.to_csv(file_path, index=False)
            driver.close()
            
        else:
            print(f"File {file_path} already exists. Skipping…\n")
    
print(f"Crawling for {dong} completed.\n")

블로그 리뷰 콘텐츠 크롤링

In [ ]:

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")

# 크롬 웹브라우저 실행
driver = webdriver.Chrome()
driver.get("https://search.naver.com")
time.sleep(1)

dong_list = ['연희동', '한남동', '망원동', '연남동', '성수동']

for dong in dong_list:
    # 폴더 경로 설정
    folder_path = f"/Users/hana/Desktop/Proj/cabojago/data/blog_review/blog_review_url_list/{dong}/"
    
    # 폴더 내 파일 목록 가져오기
    file_list = os.listdir(folder_path)
    
    # 파일 순차적으로 읽기
    for file_name in file_list:
        cafe_name = file_name.split('_')[0]
        
        
        # 결과 저장 파일 경로 읽기 
        result_folder_path = f"/Users/hana/Desktop/Proj/cabojago/data/blog_review/blog_review_result_list/{dong}/"
        in_result_file_path = os.path.join(result_folder_path, f"{cafe_name}_blog_content.csv")
        
        if not os.path.exists(in_result_file_path):
            print(file_name)
        
            # 파일 읽기 예시 (pandas를 사용하여 CSV 파일을 읽는 경우)
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                
                result_dict = {}
    
                number = len(df)
                
                for i in tqdm_notebook(range(number)):
                    # 글 띄우기
                    url = df['url'][i]
                    driver.get(url)
                    
                    try:
                        driver.switch_to.frame('mainFrame')
                
                        target_info = {}
                
                        # 제목 크롤링
                        overlays = ".se-module.se-module-text.se-title-text"
                        tit = driver.find_element(By.CSS_SELECTOR, overlays)
                        title = tit.text
                
                        # 내용 크롤링
                        overlays = ".se-main-container"
                        content = driver.find_element(By.CSS_SELECTOR, overlays)
                        cont = content.text
                        cont = cont.replace('\n', ' ')
                
                        # 딕셔너리에 담음
                        target_info['title'] = title
                        target_info['content'] = cont
                
                        # 전체 딕셔너리에 담음
                        result_dict[i] = target_info
                        time.sleep(1)
                
                        print(i, title)

                    except:
                        continue
                    
                print('수집글 갯수', len(result_dict))
                result = pd.DataFrame.from_dict(result_dict, orient='index')

                # 결과 저장 폴더가 존재하지 않으면 생성
                if not os.path.exists(result_folder_path):
                    os.makedirs(result_folder_path)

                # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
                result_file_name = f"{cafe_name}_blog_content.csv"
                
                # 파일 전체 경로 생성
                result_file_path = os.path.join(result_folder_path, result_file_name)
                
                # 데이터프레임을 CSV 파일로 저장
                result.to_csv(result_file_path, index=False)
                print(" ")
                time.sleep(3)

        else:
            print(f"File {in_result_file_path} already exists. Skipping…\n")

# 마무리 작업
driver.close()
print("All files processed.")
